# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (80, 100)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 80 to 100:
jerry: yes! 

george: youre a back-up, youre a second-line, a just-in-case, a b-plan, a contingency! 

jerry: oh, i get it, this is about the button. 

george: claire, claire, youre a woman, right? 

claire: what gave it away, george? 

george: uhm...id like to ask you...ask you to analyze a hypothetical phone call, you know, from a female point of view. 

george: (to claire) now, a woman calls me, all right? 

claie: uh huh. 

george: she says she has to come to new york on business... 

jerry: oh you are beautiful! 



---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    # key: word
    # value: frequency
    counter = Counter(text)
#     print(counter)
    # smaller value -> frequent key (word)
    vocab = sorted(counter, key=counter.get, reverse=True)
#     print(vocab)
    vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}  # index starting from 1
    # reverse the key, value
    int_to_vocab = {v: k for k, v in vocab_to_int.items()}
#     print(int_to_vocab)
    
    # return tuple
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [46]:
import collections as ct

def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    '''
    punctuation_list = [".", ",", "\"", ";", "!", "?", "(", ")", "-", "\n"]
    punctuation = {p: token for p, token in ct.Counter(text).items() if p in punctuation_list}
#     print(punctuation)
    for k, v in punctuation.items():
        punctuation[k] = str(v)  # make value type as str
#     print(punctuation)
    return punctuation
    '''
    
    puncutation = {
        ".": "||Period||",
        ",": "||Comma||",
        "\"": "||Quotation_Mark||",
        ";": "||Semicolon||",
        "!": "||Exclamation_Mark||",
        "?": "||Question_Mark||",
        "(": "||Left_Parentheses||",
        ")": "||Right_Parentheses||",
        "-": "||Dash||",
        "\n": "||Return||"
    }
    return puncutation

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [47]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [48]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [49]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [50]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    features = []
    targets = []
    for i in range(len(words) - sequence_length - 1):
        sample = words[i:i+sequence_length]
        target = words[i+sequence_length]
        features.append(sample)
        targets.append(target)
    features = np.array(features)
    targets = np.array(targets)
    feature_tensors = torch.from_numpy(features)
    target_tensors = torch.from_numpy(targets)
    
    dataset = TensorDataset(feature_tensors, target_tensors)
    dataloader = DataLoader(dataset, shuffle=True, batch_size=batch_size)
    
    # return a dataloader
    return dataloader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [51]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 16,  17,  18,  19,  20],
        [ 43,  44,  45,  46,  47],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [  3,   4,   5,   6,   7],
        [ 25,  26,  27,  28,  29],
        [ 32,  33,  34,  35,  36],
        [ 21,  22,  23,  24,  25],
        [ 36,  37,  38,  39,  40],
        [ 22,  23,  24,  25,  26]])

torch.Size([10])
tensor([ 21,  48,  39,  16,   8,  30,  37,  26,  41,  27])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [60]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        
        # set class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define model layers
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.dropout = nn.Dropout(p=0.3)
#         self.sig = nn.Sigmoid()
        
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function   
        
        # return one batch of output word scores and the hidden state
        batch_size = nn_input.size(0)
#         print('batch_size =', batch_size)
        
        embeds = self.embed(nn_input)  # output: embedding
        lstm_output, hidden_state = self.lstm(embeds, hidden)  # output: lstm_out, new hidden state
        
        # stack the output of lstm
        lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)
        
        # feed to fully-connected layer
        output = self.dropout(lstm_output)
        output = self.fc(output)
        
        # pass to sigmoid   Note: No sigmoid is required, because we want to output the socres for the next word
#         sigmoid_out = self.sig(output)  
#         print('sigmoid_out.shape:', sigmoid_out.shape)
        
        # reshape into (batch_size, seq_length, output_size)
        output = output.view(batch_size, -1, self.output_size)
#         print('sigmoid_out.shape:', sigmoid_out.shape)
        # get the sigmoid_out on the last batch
        output = output[:, -1]
        
        return output, hidden_state
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        
        # initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data
        
        if train_on_gpu:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [75]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    
    # move data to GPU, if available
    if train_on_gpu:
        inp = inp.cuda()
        target = target.cuda()
#         rnn.cuda()
        
    gradient_clip = 5
    
    # perform backpropagation and optimization
    # 1. get output and hidden state
    h = tuple([each.data for each in hidden])
    output, hidden_state = rnn(inp, h)
    # 2. calculate loss
#     print(output.squeeze().shape, target.shape)
#     target = target.float()  # this is not necessary, because criterion expectes a LongTensor not FloatTensor
#     print(target.type())
    loss = criterion(output.squeeze(), target)
#     print(loss.type())
    # 3. clear gradient and calculate new gradient
    optimizer.zero_grad()
    loss.backward()
    nn.utils.clip_grad_norm_(rnn.parameters(), gradient_clip)  # gradient clipping to avoid exploding gradient
    optimizer.step()
    
    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden_state

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [69]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [88]:
# Data params
# Sequence Length
sequence_length = 10  # of words in a sequence
# Batch Size
batch_size = 200

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [89]:
# Training parameters
# Number of Epochs
num_epochs = 4
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int) + 1 
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 400
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [90]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [91]:
# training the model
from workspace_utils import active_session

with active_session():
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)


Training for 4 epoch(s)...
Epoch:    1/4     Loss: 5.390097703933716

Epoch:    1/4     Loss: 4.802493225097656

Epoch:    1/4     Loss: 4.5922645268440245

Epoch:    1/4     Loss: 4.47483894109726

Epoch:    1/4     Loss: 4.389159416675567

Epoch:    1/4     Loss: 4.316618631839752

Epoch:    1/4     Loss: 4.2956796636581425

Epoch:    1/4     Loss: 4.250500675201416

Epoch:    2/4     Loss: 4.156462499499321

Epoch:    2/4     Loss: 4.11617911863327

Epoch:    2/4     Loss: 4.099594977855682

Epoch:    2/4     Loss: 4.087799982070923

Epoch:    2/4     Loss: 4.066811373710633

Epoch:    2/4     Loss: 4.053813449859619

Epoch:    2/4     Loss: 4.058622104167938

Epoch:    2/4     Loss: 4.043810043811798

Epoch:    3/4     Loss: 3.9665390541156134

Epoch:    3/4     Loss: 3.9251154589653017

Epoch:    3/4     Loss: 3.9270482459068297

Epoch:    3/4     Loss: 3.9340152096748353

Epoch:    3/4     Loss: 3.9197811102867126

Epoch:    3/4     Loss: 3.928274398803711

Epoch:    3/4     Loss

*This cell is for the purpose of saving the notebook progress while training.*

.
..
...



In [92]:
# saving the trained model
helper.save_model('./save/trained_rnn__after_4_epochs', trained_rnn)
print('Model Trained and Saved -> ./save/trained_rnn__after_4_epochs.pt')
torch.save(rnn.state_dict(), './save/trained_rnn_dict__after_4_epochs.pt')
print('Model Trained and Saved -> ./save/trained_rnn_dict__after_4_epochs.pt')

Model Trained and Saved -> ./save/trained_rnn__after_4_epochs.pt
Model Trained and Saved -> ./save/trained_rnn_dict__after_4_epochs.pt


/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [93]:
# continue to train

# 1. re-define parameters, optimizer, dataloader
# Data params
# Sequence Length
sequence_length = 10     # use same sequence length
# Batch Size
batch_size = 200         # use same batch size
# Training parameters
# Number of Epochs
num_epochs = 4           # use same training epochs
# Learning Rate
learning_rate = 0.001    # use te same learning rate
# re-defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
# redefine data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [94]:
# 2. continue to train
# training the model
with active_session():
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

Training for 4 epoch(s)...
Epoch:    1/4     Loss: 3.8145217185020446

Epoch:    1/4     Loss: 3.8926459197998047

Epoch:    1/4     Loss: 3.9059712586402893

Epoch:    1/4     Loss: 3.9338449096679686

Epoch:    1/4     Loss: 3.9086518278121947

Epoch:    1/4     Loss: 3.9227416672706603

Epoch:    1/4     Loss: 3.9336047015190125

Epoch:    1/4     Loss: 3.925935345649719

Epoch:    2/4     Loss: 3.872168933103482

Epoch:    2/4     Loss: 3.82077458524704

Epoch:    2/4     Loss: 3.8055745906829834

Epoch:    2/4     Loss: 3.798685182094574

Epoch:    2/4     Loss: 3.8237175469398497

Epoch:    2/4     Loss: 3.8265236248970034

Epoch:    2/4     Loss: 3.8184755492210387

Epoch:    2/4     Loss: 3.83486399269104

Epoch:    3/4     Loss: 3.784398428847392

Epoch:    3/4     Loss: 3.7480033116340636

Epoch:    3/4     Loss: 3.7221825027465822

Epoch:    3/4     Loss: 3.7294530730247497

Epoch:    3/4     Loss: 3.7675545196533204

Epoch:    3/4     Loss: 3.753897921562195

Epoch:    3/4 

*This cell is for the purpose of saving the notebook progress while training.*

.
..
...
....
.....



In [95]:
# saving the trained model
helper.save_model('./save/trained_rnn__after_8_epochs', trained_rnn)
print('Model Trained and Saved -> ./save/trained_rnn__after_8_epochs.pt')
torch.save(rnn.state_dict(), './save/trained_rnn_dict__after_8_epochs.pt')
print('Model Trained and Saved -> ./save/trained_rnn_dict__after_8_epochs.pt')

Model Trained and Saved -> ./save/trained_rnn__after_8_epochs.pt
Model Trained and Saved -> ./save/trained_rnn_dict__after_8_epochs.pt


/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [96]:
# continue to train

# 1. re-define parameters, optimizer, dataloader
# Data params
# Sequence Length
sequence_length = 10     # use same sequence length
# Batch Size
batch_size = 300         # use larger batch size (200 -> 300)
# Training parameters
# Number of Epochs
num_epochs = 4           # use same training epochs
# Learning Rate
learning_rate = 0.0005    # use smaller learning rate (0.001 -> 0.0005)
# re-defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
# redefine data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [97]:
# 2. continue to train
# training the model
with active_session():
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

Training for 4 epoch(s)...
Epoch:    1/4     Loss: 3.6536482520103455

Epoch:    1/4     Loss: 3.670161235809326

Epoch:    1/4     Loss: 3.69016055059433

Epoch:    1/4     Loss: 3.69636972284317

Epoch:    1/4     Loss: 3.690000548362732

Epoch:    2/4     Loss: 3.66843861494623

Epoch:    2/4     Loss: 3.6328777990341186

Epoch:    2/4     Loss: 3.652470661640167

Epoch:    2/4     Loss: 3.657142683029175

Epoch:    2/4     Loss: 3.642203453063965

Epoch:    3/4     Loss: 3.6191879222963967

Epoch:    3/4     Loss: 3.588239978313446

Epoch:    3/4     Loss: 3.606030616283417

Epoch:    3/4     Loss: 3.6093497548103333

Epoch:    3/4     Loss: 3.6333847489356996

Epoch:    4/4     Loss: 3.583686879825494

Epoch:    4/4     Loss: 3.5691786317825316

Epoch:    4/4     Loss: 3.574327736854553

Epoch:    4/4     Loss: 3.585133943080902

Epoch:    4/4     Loss: 3.5699284410476686



*This cell is for the purpose of saving the notebook progress while training.*

.
..
...
....
.....
......



In [98]:
# saving the trained model
helper.save_model('./save/trained_rnn__after_12_epochs', trained_rnn)
print('Model Trained and Saved -> ./save/trained_rnn__after_12_epochs.pt')
torch.save(rnn.state_dict(), './save/trained_rnn_dict__after_12_epochs.pt')
print('Model Trained and Saved -> ./save/trained_rnn_dict__after_12_epochs.pt')

Model Trained and Saved -> ./save/trained_rnn__after_12_epochs.pt
Model Trained and Saved -> ./save/trained_rnn_dict__after_12_epochs.pt


/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [99]:
# continue to train

# 1. re-define parameters, optimizer, dataloader
# Data params
# Sequence Length
sequence_length = 10     # use same sequence length
# Batch Size
batch_size = 300         # use larger batch size (200 -> 300)
# Training parameters
# Number of Epochs
num_epochs = 4           # use same training epochs
# Learning Rate
learning_rate = 0.0005    # use smaller learning rate (0.001 -> 0.0005)
# re-defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
# redefine data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [100]:
# 2. continue to train
# training the model
with active_session():
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

Training for 4 epoch(s)...
Epoch:    1/4     Loss: 3.54882301568985

Epoch:    1/4     Loss: 3.6002959308624267

Epoch:    1/4     Loss: 3.6197392954826353

Epoch:    1/4     Loss: 3.6189977641105653

Epoch:    1/4     Loss: 3.6433065237998963

Epoch:    2/4     Loss: 3.598299851520456

Epoch:    2/4     Loss: 3.563654462814331

Epoch:    2/4     Loss: 3.5904565377235413

Epoch:    2/4     Loss: 3.6006718621253966

Epoch:    2/4     Loss: 3.6008693900108337

Epoch:    3/4     Loss: 3.56394593419176

Epoch:    3/4     Loss: 3.544987217903137

Epoch:    3/4     Loss: 3.5610100989341738

Epoch:    3/4     Loss: 3.5701802687644957

Epoch:    3/4     Loss: 3.5588522243499754

Epoch:    4/4     Loss: 3.547399387330215

Epoch:    4/4     Loss: 3.5325204491615296

Epoch:    4/4     Loss: 3.522515039920807

Epoch:    4/4     Loss: 3.522882445335388

Epoch:    4/4     Loss: 3.548902366638184



*This cell is for the purpose of saving the notebook progress while training.*

.
..
...
....
.....
......
.......
........



In [101]:
# saving the trained model
helper.save_model('./save/trained_rnn__after_16_epochs', trained_rnn)
print('Model Trained and Saved -> ./save/trained_rnn__after_16_epochs.pt')
torch.save(rnn.state_dict(), './save/trained_rnn_dict__after_16_epochs.pt')
print('Model Trained and Saved -> ./save/trained_rnn_dict__after_16_epochs.pt')

Model Trained and Saved -> ./save/trained_rnn__after_16_epochs.pt
Model Trained and Saved -> ./save/trained_rnn_dict__after_16_epochs.pt


/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [103]:
# continue to train

# 1. re-define parameters, optimizer, dataloader
# Data params
# Sequence Length
sequence_length = 10     # use same sequence length
# Batch Size
batch_size = 400         # use larger batch size (300 -> 400)
# Training parameters
# Number of Epochs
num_epochs = 8           # use more epochs (4 -> 8)
# Learning Rate
learning_rate = 0.0001   # use smaller learning rate (0.0005 -> 0.0001)
# re-defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
# redefine data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [104]:
# 2. continue to train
# training the model
with active_session():
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

Training for 8 epoch(s)...
Epoch:    1/8     Loss: 3.4925587110519407

Epoch:    1/8     Loss: 3.4966801981925966

Epoch:    1/8     Loss: 3.495172082901001

Epoch:    1/8     Loss: 3.4953614773750306

Epoch:    2/8     Loss: 3.4923069921258376

Epoch:    2/8     Loss: 3.49282848072052

Epoch:    2/8     Loss: 3.4813193049430846

Epoch:    2/8     Loss: 3.476903322696686

Epoch:    3/8     Loss: 3.473225096480487

Epoch:    3/8     Loss: 3.468127067565918

Epoch:    3/8     Loss: 3.481319321632385

Epoch:    3/8     Loss: 3.479913770675659

Epoch:    4/8     Loss: 3.4545792504532695

Epoch:    4/8     Loss: 3.4702400703430176

Epoch:    4/8     Loss: 3.4637949557304384

Epoch:    4/8     Loss: 3.476693449020386

Epoch:    5/8     Loss: 3.4617343896055877

Epoch:    5/8     Loss: 3.467666127681732

Epoch:    5/8     Loss: 3.4666227746009826

Epoch:    5/8     Loss: 3.4459949798583986

Epoch:    6/8     Loss: 3.447485142864593

Epoch:    6/8     Loss: 3.453982481479645

Epoch:    6/8    

*This cell is for the purpose of saving the notebook progress while training.*

.
..
...
....
.....
......
.......
........
.........
..........
...........


In [105]:
# saving the trained model
helper.save_model('./save/trained_rnn__after_24_epochs', trained_rnn)
print('Model Trained and Saved -> ./save/trained_rnn__after_24_epochs.pt')
torch.save(rnn.state_dict(), './save/trained_rnn_dict__after_24_epochs.pt')
print('Model Trained and Saved -> ./save/trained_rnn_dict__after_24_epochs.pt')

Model Trained and Saved -> ./save/trained_rnn__after_24_epochs.pt
Model Trained and Saved -> ./save/trained_rnn_dict__after_24_epochs.pt


/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** (Write answer, here)

I tried several times on different `batch_size=50, 200, 300, 400`. I found training loss deacreases much faster when `batch_size=200`. Only after 4 epochs, loss only reaches to around __4.0__ if `batch_size=50`, however, loss decreases to __3.8__ if `batch_size=200` which is really close to our target.

For other parameters, `learning rate`, `embedding_dim`, `n_layer`, `hidden_dim`, I refer to the settings in *Sentiment_RNN_Exercise* notebook and use similar values and tuned a little bit, since I found the results in sentiment analysis task was quite well.

In total, I trained 6 x 4 epochs, where the first 4 + 4 epochs use the same parameters `batch_size=200, learning rate = 0.001`, and the following 4 + 4 epochs use the parameters `batch_size=300, learning rate = 0.0005`. Finally, the last 8 epochs use the parameter `batch_size=400, learning rate=0.0001`. At the end, the training loss is `3.445261229991913`.



---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [106]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
# trained_rnn = helper.load_model('./save/trained_rnn')
trained_rnn = helper.load_model('./save/trained_rnn__after_24_epochs')


## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [107]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [114]:
# run the cell multiple times to get different results!
gen_length = 300 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: i think it's not a good idea.

jerry: what about the meeting?

jerry: well, you know what you mean to say?!

jerry: what?

jerry:(pointing to his head) yeah, i got the same news for a while.

elaine: well, i was going to get out of this city.

kramer: well, i think i can go to the bathroom.

kramer: oh, i don't know what to do.

kramer: oh! yeah, i know what it is.

george: what?

jerry: yeah.

jerry:(to elaine) hey, you know what?

jerry: well, it's a big salad.

george:(to elaine) i can't believe i was going to have it.

george: i know.

jerry: what are you doing here?

kramer: i don't want to be able to do it.

jerry: well, it's a little bit of the...

jerry: oh, yeah, i think it's a good thing i have to say.

jerry: what?

elaine: i thought he was...

kramer:(quietly) yeah, well, you know, i think i was going to do something.

jerry: what?

kramer:(looking at jerry) i know, i'm going to get to the hospital.

jerry: oh, yeah.

jerry: what do you mean?

jerry: i know


#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [115]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.